<a href="https://colab.research.google.com/github/gaespa007648/keybert-zhTW/blob/main/keybert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install keybert
!pip install ckip_transformers
!pip install gql

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cache

In [4]:
from gql.transport.aiohttp import AIOHTTPTransport
from gql import gql, Client
from ckip_transformers.nlp import CkipWordSegmenter
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup as bs

In [1]:
### 你需要在Colab當中設定GQL_ENDPOINT並將其讀入
from google.colab import userdata
GQL_ENDPOINT = userdata.get('GQL_ENDPOINT')

In [ ]:
def gql_fetch(gql_endpoint, gql_string):
    gql_transport = AIOHTTPTransport(url=gql_endpoint)
    gql_client = Client(transport=gql_transport,
                        fetch_schema_from_transport=True)
    json_data = gql_client.execute_async(gql(gql_string))
    return json_data

gql_externals = '''
query Externals {{
  externals(where: {{state: {{equals: "published"}} }}, take: {take}, orderBy: {{publishedDate: desc}}) {{
    id
    state
    publishedDateString
    title
    partner {{
      id
      name
    }}
    thumb
    source
    brief
    content
  }}
}}
'''

In [ ]:
### 載入ckiptransformer
model = 'bert-base'
ws  = CkipWordSegmenter(model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/407M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def ws_tokenizer(text):
  words = ws([text])
  return words[0]

vectorizer = CountVectorizer(tokenizer=ws_tokenizer)

### 測試文章關鍵詞

In [ ]:
docs="藝人黃子佼性騷風波連環爆，近期又因持有7部未成年不雅片、遭K小姐指控未成年性侵，引爆大眾怒火。歌手「阿扣」謝和弦曾公開表示很欣賞黃子佼，2017年時還在手上留下「黃紫蕉」刺青，今（11）晚他特地到刺青店去除刺青，透過行動表達反對立場。謝和弦曾公開表示很欣賞黃子佼，出道後常收到黃子佼訊息打氣，當時還發文大讚：「我很欣賞跟佩服的一個台灣主持人，他剛好也是牡羊座，也從谷底爬起來過，刺上黃色紫色的香蕉，他叫做黃子佼。」"

In [ ]:
kw_model = KeyBERT(model='distiluse-base-multilingual-cased-v1')

In [ ]:
keywords = kw_model.extract_keywords(docs, vectorizer=vectorizer)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Inference: 100%|██████████| 1/1 [00:00<00:00,  1.45it/s]


In [ ]:
keywords

[('黃子佼', 0.3041),
 ('黃紫蕉', 0.2894),
 ('藝人', 0.1648),
 ('刺青店', 0.1576),
 ('黃色', 0.1434)]

### 使用週刊文章進行測試

In [ ]:
gql_externals_string = gql_externals.format(take=50)
externals = await gql_fetch(GQL_ENDPOINT, gql_externals_string)
externals = externals['externals']
posts = [post['title']+post['brief']+post['content'] for post in externals]

In [ ]:
class KeywordExtractor():
  def __init__(self, model, vectorizer):
    self.kw_model  = KeyBERT(model=model)
    self.vectorizer = vectorizer
  def preprocessing(self, posts):
    '''
    Remove html tag and punctuations
    '''
    for idx, post in enumerate(posts):
      soup = bs(post, 'html.parser')
      for a_tag in soup.find_all('a'):
        a_tag.extract()
      posts[idx] = soup.get_text()
    return posts
  def get_keywords(self, posts):
    keywords_table = []
    posts = self.preprocessing(posts)
    for post in posts:
      keywords = kw_model.extract_keywords(post, vectorizer=vectorizer)
      keywords_table.append(keywords)
    return keywords_table

In [ ]:
keyword_extractor = KeywordExtractor(
    model='distiluse-base-multilingual-cased-v1',
    vectorizer=vectorizer
)

In [ ]:
result = keyword_extractor.get_keywords(posts[:10])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Inference: 100%|██████████| 1/1 [00:04<00:00,  4.03s/it]
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Inference: 100%|██████████| 1/1 [00:06<00:00,  6.41s/it]
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Inference: 100%|██████████| 1/1 [00:03<00:00,  3.50s/it]
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Inference: 100%|██████████| 1/1 [00:03<00:00,  3.36s/it]
/usr/local/lib/pytho

In [ ]:
def preprocessing(post):
  soup = bs(post, 'html.parser')
  for a_tag in soup.find_all('a'):
    a_tag.extract()
  return soup.get_text()

check_idx = 0
print(preprocessing(posts[check_idx]))
print('關鍵詞:', result[check_idx])


 乳癌術後淋巴水腫脹痛   體外震波助安心入睡 

 好醫師新聞網記者邱秉維／桃園報導

圖：體外震波操作示意／聯新國際醫院提供。
一名40歲女性因乳癌晚期接受左側乳房切除，及腋下淋巴節根除手術，造成左上肢淋巴水腫疼痛無法入睡，只好求助醫師，醫師為她實行體外震波儀治療，並持續自主復健，終於消除水腫，安心入睡。
 
聯新國際醫院復健科主治醫師朱怡妍表示，該女因乳癌第三期，執行左側乳房切除併單側淋巴節根除術，並且接受化療及放射線治療，術後新陳代謝不順，導致淋巴水腫，更有肩膀疼痛，讓她夜晚不能入睡。「淋巴水腫在乳癌術後不算少見，要透過淋巴引流，才能減少腫脹」朱怡妍說。
 
如何將淋巴引流呢？整合性的方式包含徒手引流、壓力繃帶的使用、運動、皮膚照護，並輔助使用體外震波，或是使用復健的循環機改善血液新陳代謝。朱怡妍指出，「較好方式是接受『雙系統體外震波治療儀』治療，震波治療（又稱骨震波治療）即是透過高能量的聲波傳遞到深層的軟組織，幫助組織活化與刺激骨骼癒合等，並且能阻絕痛覺神經傳遞，達到止痛效果。」
 
聯新國際醫院物理治療師葉哲輔進一步解釋，「為了舒緩淋巴結，會以淋巴流動方向疏通淋巴結，恢復血管的新陳代謝；此外，也會讓病人練習自主復健，像是手要抬高於心臟，並練習手緊握，讓血液循環，加速新陳代謝，並接受體外震波儀的治療，透過震波治療打入病灶處，讓它產生發炎反應，藉由血管新生的作用，帶走發炎物質，用以促進組織修復與再生。」
 
體外震波治療儀可針對足底筋膜炎、跟腱炎、慢性踝關節扭傷、骨關節鈣化肌腱炎、五十肩、肩頸疼痛、媽媽手、網球肘、高爾夫球肘、腕隧道炎、扳機指、膝關節軟組織疼痛等，也能有效治療，減緩肌肉疼痛，詳細應諮詢專業醫師。





















關鍵詞: [('乳癌', 0.5322), ('淋巴水腫', 0.3126), ('淋巴節', 0.306), ('淋巴引流', 0.2833), ('乳房', 0.2764)]


In [ ]:
check_idx = 1
print(preprocessing(posts[check_idx]))
print('關鍵詞:', result[check_idx])


 健檢驗出RF陽性  不代表罹患類風濕性關節炎 

 好醫師新聞網記者邱秉維／新竹報導

圖：林冠言醫師聽到這類疑問時，都會耐心說明／新竹台大醫院提供
林小姐隨著公司同事一起接受健檢，但是看到報告後卻滿腹疑問的請教醫師：「醫生，這次健康檢查報告顯示類風性關節因子（Rheumatoid factor，以下簡稱RF）呈陽性，是不是得了類風溼性關節炎(Rheumatoid Arthritis，以下簡稱RA)？」。新竹臺大分院新竹醫院內科部林冠言醫師聽到這類疑問時，都會耐心說明RF及類風溼性關節炎兩者間雖具有一定的關聯性，但實際上， RF陽性並不一定代表罹患類風溼性關節炎。
 
林冠言醫師表示，RF是一種免疫球蛋白M（IgM）抗體，最初是在1940年代被發現。大量的研究已經確立了RF在類風溼性關節炎診斷和監測中的重要性；某些研究指出，當RF濃度超過正常值的2至3倍以上時，與RA的關聯性可能進一步增強。另外，也有部分研究報告提到，在類風溼性關節炎的病人中，RF的敏感性約為70%，但特異性也僅有70%。因此要確定是否為類風濕性關節炎，通常醫師會結合病人的症狀（如是否有關節疼痛、腫脹、晨僵感以及對稱性的關節受影響等）、身體檢查結果、其他血液測試（如抗瓜胺酸蛋白抗體，簡稱CCP抗體）、C-反應蛋白檢測（簡稱CRP）、血沉測試（簡稱ESR）和影像學檢查等多方面的資料進行綜合評估。
 
林冠言醫師解釋，RF陽性結果除了與類風溼性關節炎相關外，還可能與以下狀況或疾病有關：
1. 健康人口也可能呈現陽性：這被視為正常的現象。有研究顯示約有5至10％的健康人口呈現RF陽性，甚至隨著年齡的增加，陽性的比率還會略為上升。在一些老年人群的研究中，RF陽性率可能達到10%以上，甚至在某些研究中超過20%。這些數字可能受到研究參與者的選擇、地理位置、研究設計和RF檢測方法等多方面因素的影響。
2. 其他自體免疫疾病：如紅斑狼瘡、修格蘭氏症候群等，也可能伴隨RF的陽性。
3. 感染性疾病： 某些感染性疾病，特別是慢性感染，可能導致RF的陽性。這包括病毒性肝炎、結核病、梅毒等。
4. 慢性肝病：特別是肝硬化，與RF的陽性結果之間存在一定的相關性。
 
林冠言醫師提醒，如果RF檢測結果呈陽性，請不要驚慌，但也不要忽視。盡快諮詢專業的風濕病專科醫生的建議及進行全面的評估，才是確定診斷和制定適當治療計劃的重要

In [ ]:
check_idx = 5
print(preprocessing(posts[check_idx]))
print('關鍵詞:', result[check_idx])

熱心公益助弱勢　劉柱均當選青年代表阜冠國際開發有限公司總經理劉柱均，自父親手中接下傳統成衣產業並進行轉型，創立「阜冠」品牌，堅持台灣製造，提供舒適、優質產品，並持續參與兒福、教育、動物保育等相關公益活動，希望能夠為社會做出一點貢獻，幫助需要幫助的人，獲得今年救國團高雄市青年節社會優秀青年代表實至名歸。生活中心／洪正達報導

▲劉柱均創業後，也扶助許多弱勢進而當選青年代表。（圖／翻攝畫面）
阜冠國際開發有限公司總經理劉柱均，自父親手中接下傳統成衣產業並進行轉型，創立「阜冠」品牌，堅持台灣製造，提供舒適、優質產品，並持續參與兒福、教育、動物保育等相關公益活動，希望能夠為社會做出一點貢獻，幫助需要幫助的人，獲得今年救國團高雄市青年節社會優秀青年代表實至名歸。
劉柱均的雙親早年在武廟市場發跡，從擺攤賣衣服開始，後來經人牽線走入成衣製造行業，一開始幫廠商做外套一砲而紅，也接了大型運動會及不少選舉夾克的生意，成為高雄最大的外套供應商，當時還屬於家庭代工性質，成員也只有三人。
中國開放台商登陸，低廉人工成本吸引許多成衣廠外移，技術及幹部都被帶走，最後成品還大量回銷台灣，一下子打壞外套市場行情，劉柱均的雙親被迫轉型做polo衫，反而闖出一條生路。劉柱均說父親天生對色差的敏銳度極強，可以調配出坊間所沒有的顏色，獲得客戶青睞及信任。

▲劉柱均（後排左四）贊助嘉興國中巧固球隊服裝。（圖／翻攝畫面）
為了分攤二老的辛勞，劉柱均服完兵役立即投入家族事業，並進行一連串企業改革，從紙本進入電子化，並改良管理系統，讓公司體質更趨完善，自己也繼續進修學習電腦繪圖，跟上時代潮流，也踏出奠定公司現代化的穩健腳步。
劉柱均表示，從上一代身上學到許多，包括追求完美的精神，不好的產品絕對不會拿到市場上，對顧客也永遠堅持最好的服務及負責到底的態度。他不只聽經銷商的聲音，也直接聽末端使用者的需求，甚至連數量極少的訂單都接，遭同行質疑甚至訕笑，最後證明他是對的，不但補上市場上的缺口，也不會碰到客戶削價的問題，營業額成長不少，在長期耕耘下，一些發展快速的連鎖品牌也成為死忠支持者。目前公司已達到一定規模，接下來將進入擴編階段，主打外銷市場，以「台灣製造」為號召。
這幾年劉柱均和其他企業共同贊助一支在地的自由車隊，陣中扛霸子亞洲車后黃亭茵，車隊曾參與彭于晏電影「破風」的演出，去年三家贊助商相繼退出，車隊教練找上劉柱均尋

In [ ]:
check_idx = 7
print(preprocessing(posts[check_idx]))
print('關鍵詞:', result[check_idx])

曾欣賞黃子佼！謝和弦刺青「黃紫蕉」後悔了　塗黑蓋掉曝真實原因
藝人黃子佼性騷風波連環爆，近期又因持有7部未成年不雅片、遭K小姐指控未成年性侵，引爆大眾怒火。歌手「阿扣」謝和弦曾公開表示很欣賞黃子佼，2017年時還在手上留下「黃紫蕉」刺青，今（11）晚他特地到刺青店去除刺青，透過行動表達反對立場。




謝和弦曾公開表示很欣賞黃子佼，出道後常收到黃子佼訊息打氣，當時還發文大讚：「我很欣賞跟佩服的一個台灣主持人，他剛好也是牡羊座，也從谷底爬起來過，刺上黃色紫色的香蕉，他叫做黃子佼。」

黃子佼去（2023）年爆發性騷事件後，一度試圖輕生，當時謝和弦發文鼓勵他「佼哥沒事的，請你好好活下去」、「雖然不能功過相抵，但是我們不會忘記」。





如今黃子佼風波越演越烈，甚至被發現涉及未成年性犯罪，謝和弦除掉刺青之際，更表示「為了女兒，哈們」、「你懂我的黑色幽默」，老婆陳緗妮也笑說「謝音悅會以為爸爸是蝙蝠俠」。

 


《TVBS》提醒您： 
根據刑法第235條規定，散布、播送或販賣猥褻之文字、圖畫、聲音、影像或其他物品，或公然陳列，或以他法供人觀覽、聽聞者，處2年以下有期徒刑、拘役或科或併科三萬元以下罰金。意圖散布、播送、販賣而製造、持有前項文字、圖畫、聲音、影像及其附著物或其他物品者，亦同。提醒大家勿加入跪求、私給等行列以免觸法。
 


延伸閱讀：【本文由《》授權提供。】
				
關鍵詞: [('黃紫蕉', 0.3574), ('黃子佼', 0.3359), ('謝和弦', 0.2149), ('陳緗妮', 0.208), ('香蕉', 0.1866)]
